# Run experiment
- memo
    - This experiment is conducted at 2023/12/28
- The purpose of this experiment
    - HONBAN 9回目
- The specificity of this experiment
    - 2 plates
    - edge spotting will be done just after spotting



- labwere installing
    - deck1:none
    - deck2:none
    - deck3:none
    - deck4:p300 tip rack
    - deck5:p20 tip rack
    - deck6:YPD agar plate in microplate
    - deck7:Opentrons Tube rack
        - A4:yeast culture (biorep1) in 50 mL tube 3mL preculture in 25 mL fresh YPD liquid
        - B4:yesst culture (biorep2) in 50 mL tube 3mL preculture in 25 mL fresh YPD liquid
        - B3:water in 50 mL tube
    - deck8:Opentrons tube rack 24 2mL tubes
    - deck9:YPD agar plate in microplate
    - deck10-11:GEN2 Transportation system and 96well plate

# __実行前にRestartとClear All Outputsを実行すること__

####  `agar_plate_weight`と`plate_type` の設定方法
- agar_plate_weight_6 is the agar plate which is set on deck 6. <br>
- agar_plate_weight_9 is the same meaning.

In [1]:
agar_plate_weight_6, plate_type_6 = 66.13, "microplate"
agar_plate_weight_9, plate_type_9 = 85.74, "microplate"

In [2]:
import os
# Set working directory
os.chdir(path="/Users/yates/Desktop/OT2_yeast/code/OT2_protocols/experiment_microplate_spotting_20231120_AvoidEdgeEffect")
# Working directory
print("Current working directory:", os.getcwd())

path_outdir = "/Users/yates/Desktop/OT2_yeast/results"


Current working directory: /Users/yates/Desktop/OT2_yeast/code/OT2_protocols/experiment_microplate_spotting_20231120_AvoidEdgeEffect


In [3]:
# Relative position of 96-well plate (when pipetting) on Transporter GEN2
pipettingPos = 0

# Relative position of 96-well plate (when measuring absorbance byonoy) on Transporter GEN2
offset = -138.3

## Load modules

In [4]:
# Load .dotenv file
from dotenv import load_dotenv
load_dotenv(override=True)

import os
import subprocess

# Path to `labopt` directory (where `chef` directory is located)
path_labopt = os.path.expanduser(os.getenv('PATH_LABOPT'))

# Device path of Transporter GEN2 (can be checked by Arduino IDE)
gen2_DevicePath = os.getenv("GEN2_DEVICEPATH")

# SSH secret key for accessing to OT-2
OT2_SSH_KEY = os.path.expanduser(os.getenv("OT2_SSH_KEY"))



# Hostname of of OT-2
OT2_HOSTNAME = os.getenv("OT2_HOSTNAME")

# Path to labware_OT2_yeast
labware_ot2_yeast = "/Users/yates/Desktop/OT2_yeast/code/OT2_protocols/labware_OT2_yeast"

print("path_labopt:", path_labopt)
print("gen2_DevicePath:", gen2_DevicePath)
print("OT2_SSH_KEY:", OT2_SSH_KEY)
print("OT2_HOSTNAME", OT2_HOSTNAME)

path_labopt: /Users/yates/Desktop/labopt/labopt
gen2_DevicePath: /dev/cu.usbmodem11401
OT2_SSH_KEY: /Users/yates/.ssh/ot2-003_ssh_key
OT2_HOSTNAME OT2CEP20210420B10


In [5]:
# Test connection to OT-2 
command = 'ssh root@%s.local -i %s "source /etc/profile && cd /data/user_storage && ls"' % (OT2_HOSTNAME, OT2_SSH_KEY)
print(command)
subprocess.call(command, shell=True)

ssh root@OT2CEP20210420B10.local -i /Users/yates/.ssh/ot2-003_ssh_key "source /etc/profile && cd /data/user_storage && ls"
DEBUG_second_half.py
absorbance96
agar_plate_weight.csv
aiohttp
different_mix.py
dir_images
dir_videos
first_half.py
homing.py
image.jpg
jupyter-drive
labware
labware_OT2_yeast
liquid_volume.csv
liquid_volume_12well.csv
mix_test_record.csv
opentrons_data
pipetting_record_first.csv
pipetting_record_second.csv
protocol.py
run_first_half.py
run_second_half.py
second_half.py
serial_usb
spotassay
temp-file
test_calibration.py
test_exp_1.py
test_exp_1_record.csv
test_exp_2.py
test_exp_2_record.csv
test_pycode
wait_time.csv


0

In [6]:
import os
import shutil
import glob
import sys
import argparse
import numpy as np
import subprocess
from pathlib import Path
from datetime import datetime

# Load `labopt` functions
sys.path.append(path_labopt)
import saucier.saucier as saucier
import waiter.monger as monger
import chef
import suechef.abs_report.taste_report as taste_report
from calc_dilution_recipe import calc_dilution_recipe

# Define funcitons
def file_copy(path_target, path_to):
	for path in glob.glob(path_target):
		shutil.copy(path, path_to)

def file_remove(path_target):
	for path in glob.glob(path_target):
		os.remove(path)

def file_output(text, path):
	f = open(path, "w")
	f.writelines(text)
	f.close()

## Preparation

In [7]:
# Timestamp
timestamp = datetime.now().strftime("%y%m%d-%H%M%S")
print("Timestamp: ", timestamp)

# Make current output directory
path_outdir_now = os.path.join(path_outdir, timestamp)
os.makedirs(path_outdir, exist_ok=True)
os.makedirs(path_outdir_now, exist_ok=True)


Timestamp:  231228-204458


In [8]:
# Transfer labware definition
command = 'scp -r -i %s %s root@%s.local:/data/user_storage/' % (OT2_SSH_KEY,labware_ot2_yeast, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)

# Make agar plate weight csv
path_agar_plate_weight = os.path.join(path_outdir_now, 'agar_plate_weight.csv')
command = 'echo "agar_plate_weight_6,%.2f,%s" > %s' % (agar_plate_weight_6, plate_type_6, path_agar_plate_weight)
print(command)
subprocess.call(command, shell=True)

command = 'echo "agar_plate_weight_9,%.2f,%s" >> %s' % (agar_plate_weight_9, plate_type_9, path_agar_plate_weight)
print(command)
subprocess.call(command, shell=True)

scp -r -i /Users/yates/.ssh/ot2-003_ssh_key /Users/yates/Desktop/OT2_yeast/code/OT2_protocols/labware_OT2_yeast root@OT2CEP20210420B10.local:/data/user_storage/
echo "agar_plate_weight_6,66.13,microplate" > /Users/yates/Desktop/OT2_yeast/results/231228-204458/agar_plate_weight.csv
echo "agar_plate_weight_9,85.74,microplate" >> /Users/yates/Desktop/OT2_yeast/results/231228-204458/agar_plate_weight.csv


0

### Test connection to GEN2 transportation system and byonoy


In [9]:
# Measure absorbance
os.makedirs(os.path.join(path_outdir_now, "test_transporter"), exist_ok=True)

monger.monger(
            os.path.join(path_outdir_now, "test_transporter"), [620], arg_debug=False,
            arg_gen2=True, 
            arg_gen2_DevicePath=gen2_DevicePath, 
            arg_gen2_ByonoyPos=offset, 
            arg_gen2_pipettingPos=pipettingPos
            )

[2023-12-28 20:44:59] wavelengths to be measured: 620
Setting and call SBS plate to byonoy trans v2 dev class:
- DevicePath: /dev/cu.usbmodem11401
- pipettingPos: 0
- ByonoyPos: -138
/dev/cu.usbmodem11401
Connect the device system.
Connect SBS Plate To Byonoy Trans V2 Device ...
Wakeup the device...
Wakeup the device done.
Homing.
Homing Cycle start...
-> Done.
[2023-12-28 20:45:05] Device: /dev/cu.usbserial-DM6LBI47
[2023-12-28 20:45:05] Connected to absorbance96 meter
[2023-12-28 20:45:05] available wavelengths: 492,405,450,620
[2023-12-28 20:45:05] Start doing calibration for 620 nm...
[2023-12-28 20:45:05] Doing calibration: main=620, ref=-1
[2023-12-28 20:45:08] Done. result: Max optical disturbance: 71mOD, Measurement time: 2.9 seconds
[2023-12-28 20:45:08] Finished doing calibration!
[2023-12-28 20:45:12] OK: Absorbance96 meter is empty.
[2023-12-28 20:45:12] Start doing calibration for 620 nm...
[2023-12-28 20:45:12] Doing calibration: main=620, ref=-1
[2023-12-28 20:45:14] Don

### Homing arm of OT-2 to avoid some errors at First experiment

In [10]:
# Transfer OT-2 protocol
path_protocol = "homing.py"
basename_protocol = os.path.basename(path_protocol)

command = "scp -i %s %s root@%s.local:/data/user_storage" % (OT2_SSH_KEY, path_protocol, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)

command = 'ssh root@%s.local -i %s "source /etc/profile && cd /data/user_storage && opentrons_execute %s"' % (OT2_HOSTNAME, OT2_SSH_KEY, basename_protocol)
print(command)
subprocess.call(command, shell=True)

scp -i /Users/yates/.ssh/ot2-003_ssh_key homing.py root@OT2CEP20210420B10.local:/data/user_storage
ssh root@OT2CEP20210420B10.local -i /Users/yates/.ssh/ot2-003_ssh_key "source /etc/profile && cd /data/user_storage && opentrons_execute homing.py"
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
alarm/error outside hard halt: ALARM: Homing fail
alarm/error: command=M907 A0.1 B0.3 C0.3 X1.25 Y0.3 Z0.1 G4 P0.005 G28.2 X 

, resp=ALARM: Homing fail


Traceback (most recent call last):
  File "usr/lib/python3.7/site-packages/opentrons/drivers/smoothie_drivers/driver_3_0.py", line 894, in _send_command_unsynchronized
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 136, in send_command
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 170, in send_data
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 202, in _send_data
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 249, in raise_on_error
opentrons.drivers.asyncio.communication.errors.AlarmResponse: /dev/ttyAMA0: 'Received error response 'ALARM: Homing fail'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "usr/lib/python3.7/site-packages/opentrons/drivers/smoothie_drivers/driver_3_0.py", l

1

## ⏺ 録画を開始


## 🤖 Execute First_half 🤖

In [11]:
# compile first_half.py
command = "python3 compile.py first_half.py > run_first_half.py"
print(command)
subprocess.call(command, shell=True)

# Transfer OT-2 protocol
path_protocol = "./run_first_half.py"
basename_protocol = os.path.basename(path_protocol)

command = "scp -i %s %s root@%s.local:/data/user_storage" % (OT2_SSH_KEY, path_protocol, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)

python3 compile.py first_half.py > run_first_half.py
scp -i /Users/yates/.ssh/ot2-003_ssh_key ./run_first_half.py root@OT2CEP20210420B10.local:/data/user_storage


0

In [12]:
# Run OT-2 protocol
command = 'ssh root@%s.local -i %s "source /etc/profile && cd /data/user_storage && opentrons_execute %s"' % (OT2_HOSTNAME, OT2_SSH_KEY, basename_protocol)
print(command)
subprocess.call(command, shell=True)

ssh root@OT2CEP20210420B10.local -i /Users/yates/.ssh/ot2-003_ssh_key "source /etc/profile && cd /data/user_storage && opentrons_execute run_first_half.py"
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
 Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 4
 Mixing 1 times with a volume of 200.0 ul
	 Aspirating 200.0 uL from A4 of Opentrons 10 Tube Rack with Falcon 4x50 mL, 6x15 mL Conical on 7 at 290.0 uL/sec
	 Dispensing 200.0 uL into A4 of Opentrons 10 Tube Rack with Falcon 4x50 mL, 6x15 mL Conical on 7 at 290.0 uL/sec
 Mixing 1 times with a volume of 200.0 ul
	 Aspirating 200.0 uL from A4 of Opentrons 10 Tube Ra

0

In [13]:
# Download log
path_log = "pipetting_record_first.csv"
command = "scp -i %s root@%s.local:/data/user_storage/%s %s" % (OT2_SSH_KEY, OT2_HOSTNAME, path_log, path_outdir_now)
print(command)
subprocess.call(command, shell=True)

scp -i /Users/yates/.ssh/ot2-003_ssh_key root@OT2CEP20210420B10.local:/data/user_storage/pipetting_record_first.csv /Users/yates/Desktop/OT2_yeast/results/231228-204458


0

## Transfer and measure 96-well plate using Transporter GEN2
⚠️ Before run this, set Direct Current at 24.0 V 


In [14]:
# Measure absorbance
monger.monger(
            path_outdir_now, [620], arg_debug=False,
            arg_gen2=True, 
            arg_gen2_DevicePath=gen2_DevicePath, 
            arg_gen2_ByonoyPos=offset, 
            arg_gen2_pipettingPos=pipettingPos
            )

[2023-12-28 20:52:11] wavelengths to be measured: 620
Setting and call SBS plate to byonoy trans v2 dev class:
- DevicePath: /dev/cu.usbmodem11401
- pipettingPos: 0
- ByonoyPos: -138
/dev/cu.usbmodem11401
Connect the device system.
Connect SBS Plate To Byonoy Trans V2 Device ...
Wakeup the device...
Wakeup the device done.
Homing.
Homing Cycle start...
-> Done.
[2023-12-28 20:52:18] Device: /dev/cu.usbserial-DM6LBI47
[2023-12-28 20:52:18] Connected to absorbance96 meter
[2023-12-28 20:52:18] available wavelengths: 492,405,450,620
[2023-12-28 20:52:18] Start doing calibration for 620 nm...
[2023-12-28 20:52:18] Doing calibration: main=620, ref=-1
[2023-12-28 20:52:21] Done. result: Max optical disturbance: 70mOD, Measurement time: 2.9 seconds
[2023-12-28 20:52:21] Finished doing calibration!
[2023-12-28 20:52:24] OK: Absorbance96 meter is empty.
[2023-12-28 20:52:24] Start doing calibration for 620 nm...
[2023-12-28 20:52:24] Doing calibration: main=620, ref=-1
[2023-12-28 20:52:27] Don

## Calculate liquid volume based on absorbance measurement

In [15]:
path_absorbance = os.path.join(path_outdir_now, "result_620.csv")
path_liquid_volume = os.path.join(path_outdir_now, "liquid_volume.csv")

calc_dilution_recipe(path_absorbance, path_liquid_volume)

## 🤖Second half🤖

In [16]:
# compile second_half.py
command = "python3 compile.py second_half.py > run_second_half.py"
print(command)
subprocess.call(command, shell=True)

# Transfer OT-2 protocol
path_protocol = "run_second_half.py"
basename_protocol = os.path.basename(path_protocol)

command = "scp -i %s %s root@%s.local:/data/user_storage" % (OT2_SSH_KEY, path_protocol, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)

# Transfer liquid_volume.csv
command = "scp -i %s %s root@%s.local:/data/user_storage" % (OT2_SSH_KEY, path_liquid_volume, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)



# Transfer agar plate weight file
command = "scp -i %s %s root@%s.local:/data/user_storage" % (OT2_SSH_KEY, path_agar_plate_weight, OT2_HOSTNAME)
print(command)
subprocess.call(command, shell=True)

python3 compile.py second_half.py > run_second_half.py
scp -i /Users/yates/.ssh/ot2-003_ssh_key run_second_half.py root@OT2CEP20210420B10.local:/data/user_storage
scp -i /Users/yates/.ssh/ot2-003_ssh_key /Users/yates/Desktop/OT2_yeast/results/231228-204458/liquid_volume.csv root@OT2CEP20210420B10.local:/data/user_storage
scp -i /Users/yates/.ssh/ot2-003_ssh_key /Users/yates/Desktop/OT2_yeast/results/231228-204458/agar_plate_weight.csv root@OT2CEP20210420B10.local:/data/user_storage


0

In [17]:
# Run OT-2 protocol
command = 'ssh root@%s.local -i %s "source /etc/profile && cd /data/user_storage && opentrons_execute %s"' % (OT2_HOSTNAME, OT2_SSH_KEY, basename_protocol)
print(command)
subprocess.call(command, shell=True)

ssh root@OT2CEP20210420B10.local -i /Users/yates/.ssh/ot2-003_ssh_key "source /etc/profile && cd /data/user_storage && opentrons_execute run_second_half.py"
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
 Picking up tip from A2 of Opentrons 96 Tip Rack 300 µL on 4
 Transferring 1240.5 from B3 of Opentrons 10 Tube Rack with Falcon 4x50 mL, 6x15 mL Conical on 7 to A1 of Opentrons 24 Tube Rack with NEST 2 mL Screwcap on 8
	 Aspirating 300.0 uL from B3 of Opentrons 10 Tube Rack with Falcon 4x50 mL, 6x15 mL Conical on 7 at 92.86 uL/sec
	 Dispensing 300.0 uL into A1 of Opentrons 24 Tube Rack with NEST 2 mL Screwcap on 8 at 92.8

0

In [18]:
# Download log
path_log = "pipetting_record_second.csv"
command = "scp -i %s root@%s.local:/data/user_storage/%s %s" % (OT2_SSH_KEY, OT2_HOSTNAME, path_log, path_outdir_now)
print(command)
subprocess.call(command, shell=True)

scp -i /Users/yates/.ssh/ot2-003_ssh_key root@OT2CEP20210420B10.local:/data/user_storage/pipetting_record_second.csv /Users/yates/Desktop/OT2_yeast/results/231228-204458


0

### Measure Absorbance at the end of the protocol

In [19]:
# Measure absorbance
os.makedirs(os.path.join(path_outdir_now, "end_experiment"), exist_ok=True)
monger.monger(
            os.path.join(path_outdir_now, "end_experiment"), [620], arg_debug=False,
            arg_gen2=True, 
            arg_gen2_DevicePath=gen2_DevicePath, 
            arg_gen2_ByonoyPos=offset, 
            arg_gen2_pipettingPos=pipettingPos
            )

[2023-12-28 22:17:18] wavelengths to be measured: 620
Setting and call SBS plate to byonoy trans v2 dev class:
- DevicePath: /dev/cu.usbmodem11401
- pipettingPos: 0
- ByonoyPos: -138
/dev/cu.usbmodem11401
Connect the device system.
Connect SBS Plate To Byonoy Trans V2 Device ...
Wakeup the device...
Wakeup the device done.
Homing.
Homing Cycle start...
-> Done.
[2023-12-28 22:17:25] Device: /dev/cu.usbserial-DM6LBI47
[2023-12-28 22:17:25] Connected to absorbance96 meter
[2023-12-28 22:17:25] available wavelengths: 492,405,450,620
[2023-12-28 22:17:25] Start doing calibration for 620 nm...
[2023-12-28 22:17:25] Doing calibration: main=620, ref=-1
[2023-12-28 22:17:28] Done. result: Max optical disturbance: 68mOD, Measurement time: 2.9 seconds
[2023-12-28 22:17:28] Finished doing calibration!
[2023-12-28 22:17:32] OK: Absorbance96 meter is empty.
[2023-12-28 22:17:32] Start doing calibration for 620 nm...
[2023-12-28 22:17:32] Doing calibration: main=620, ref=-1
[2023-12-28 22:17:34] Don

### Result note
- All the spotting was successful

### ToDo
- HONBAN experiment


## Save snapshot of this Jupyter notebook, run_first_half.py and run_second_half.py

In [20]:
file_copy(os.path.join(os.path.abspath(""), "run_experiment.ipynb"), path_outdir_now)
file_copy(os.path.join(os.path.abspath(""), "run_first_half.py"), path_outdir_now)
file_copy(os.path.join(os.path.abspath(""), "run_second_half.py"), path_outdir_now)
